<a href="https://colab.research.google.com/github/brunovianna/DCGAN-Art-Tensorflow/blob/master/insect_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Check if GPU is enabled

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
print (tf.version.VERSION)

1.15.0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


check how much memory this gpu has

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

DON'T RUN - only to remove everything
💀💀💀💀💀💀💀💀
!kill -9 -1
💀💀💀💀💀💀💀💀


In [0]:
#!kill -9 -1

# Install repo and import images

Lets use the repo made by carpedm20, a tensorflow implementation to Dgans

In [0]:
!git clone https://github.com/harrywang/DCGAN-Art-Tensorflow.git

Cloning into 'DCGAN-Art-Tensorflow'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 67 (delta 11), reused 17 (delta 6), pack-reused 42
Unpacking objects: 100% (67/67), done.


In [0]:
cd DCGAN-Art-Tensorflow

/content/DCGAN-Art-Tensorflow/DCGAN-Art-Tensorflow


In [0]:
# install the necessary libraries
!pip install tqdm
!pip install -U -q PyDrive
!pip install googledrivedownloader


Import from your Google Drive your dataset -* never upload with the files.upload(), it crashes if file is too big*

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("all right")

# Import Dataset

Some of the dataset i'm using, not sure they will work on other computers, just check or else upload a folder with the same img size and colors (RGB, not RGBA) to your drive. Make it a sharable link and copy the last part like this: drive.google.com/open?id=**1LWolfUnkoAwHnxmyOz3PnI1gWsQI-oUc**

In [0]:
#link_name ="1XKrAkCV4dzVNegPk2tYJ2SxBsDATufnG" #flowers
link_name ="1O1JItVJm2nehIbQucTHP5Sct0-DZKOLO" #insects
#link_name ="1A3fVxHiH9OlGVIN5ciAixr8YEn3lBAdR" #naked
#link_name = "1xuuqhbE0OH0SEPC-pymbnzoffYubkH6t" #portrait
#link_name ="1nv7WmFpz6TPwerNG4srjz-COfVgpe2Sy" #calligraphy
#link_name ="1yHSggsUgtW4OV6k6lfaJmJMdEwnMaCmU"
#link_name="174dn8Ts6pyRwU1vNu0-0eNKZBEkbautj"
#link_name ="1aPcBcuDt4fAKemSMwhLNiFgHC8-APRe4" # nurse

create dirs to place dataset

In [0]:
mkdir data

In [0]:
mkdir data/insects

In [0]:
#rm -R data

import from drive and unzip the  file to *dest*

In [0]:
dest ="./data/insects/"+ link_name+".zip"


In [0]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id=link_name,
                                    dest_path=dest,
                                    unzip=True)

Unzipping...Done.


# Import checkpoints from drive

only if you already have them

In [0]:
link1 = "1SOB3BunfS95sHRzXFky3ZJ_nOwoX-KpG"

In [0]:
mkdir checkpoint

In [0]:
loc ="./checkpoint/"+ link1+".zip"

In [0]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id=link1,
                                    dest_path=loc,
                                    unzip=True)

In [0]:
import os
os.listdir("./checkpoint")

In [0]:
rm -r ./checkpoint/1SOB3BunfS95sHRzXFky3ZJ_nOwoX-KpG.zip

In [0]:
rm -r ./checkpoint/checkpoint/checkpoint/1SOB3BunfS95sHRzXFky3ZJ_nOwoX-KpG.zip

In [0]:
import os
os.listdir("./checkpoint")

In [0]:
import shutil
shutil.move("./checkpoint/checkpoint/naked_out_128_108_108", "./checkpoint/naked_out_128_108_108" )

# Uniform images

Run this to avoid this error: *ValueError: could not broadcast input array from shape.. *

It converts all images in a directory to RGB (it removes the alpha channel) and moves everything to a new folder

In [0]:
mkdir ./data/insects_new

In [0]:
import os
os.listdir('./data/insects/')

In [0]:
from PIL import Image
import os, sys

path ='./data/insects/'
out="./data/insects_new/"

dirs = os.listdir( path )

n=0
for item in dirs:
    try:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            longer_side = max(im.size)

            horizontal_padding = (longer_side - im.size[0]) / 2
            vertical_padding = (longer_side - im.size[1]) / 2
            f, e = os.path.splitext(path+item)
            imResize = im.crop(
            (
                -horizontal_padding,
                -vertical_padding,
                im.size[0] + horizontal_padding,
                im.size[1] + vertical_padding
            )
            )
            RGB = imResize.convert('RGB')
            little = RGB.resize((256,256), Image.ANTIALIAS)

            little.save(out +  str(n) +'resize.jpg', 'JPEG', quality=90)
            n+=1
            
    except Exception as e:
        print(e)

cannot identify image file './data/insects/1O1JItVJm2nehIbQucTHP5Sct0-DZKOLO.zip'


In [0]:
import os
os.listdir('./data/insects_new')

# Train - Test

In [0]:
cd DCGAN-Art-Tensorflow

In [0]:
!ls


In [0]:
import time
!python ./main.py  --batch_size 16 --dataset insects_new  --epoch 25  --input_fname_pattern "*resize.jpg" --input_height 256 --output_height 256  --option 0 --train  --generate_test_images=10

ts = time.asctime() + ".tar.gz"
ts = ts.replace(" ","_")
ts = ts.replace(":","")
!tar -czvf "$ts" samples
!cp "$ts" "/content/drive/My Drive/Colab Notebooks"

In [0]:
from google.colab import files
files.download( "./samples/test_gif_17.gif" )
#print("AAAH right")

In [0]:
!python main.py    --dataset insects_new   --input_fname_pattern "*resize.jpg" --input_height 256 --output_height 256  --option 0
import time
ts = time.asctime() + ".tar.gz"
ts = ts.replace(" ","_")
ts = ts.replace(":","")
!tar -czvf "$ts" samples
!cp "$ts" "/content/drive/My Drive/Colab Notebooks"


In [0]:
!cd ./DCGAN-Art-Tensorflow/

In [0]:
import os
os.listdir('./samples')

In [0]:
!rm -R checkpoint/

In [0]:
!ls -la samples/.ipynb_checkpoints/


# Save files

save specific img

In [0]:
from google.colab import files
files.download( "./samples/test_gif_13.gif" )
print("aaaa right")

aaaa right


save checkpoint folder - to continue when the VM  changes

In [0]:
cd ..

# Save to Drive - Zip file


In [0]:
CPT_PATH = "./checkpoint"

Same logic as before, this time it saves the .zip to a specific drive folder 

In [0]:
drive_folder = "116OGJqQpksx6AaRpcH5EUt2M99wJ9mfJ" # change the link, or it will fill my drive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("all right")

In [0]:
import os
os.listdir('.')

In [0]:
import zipfile
import os
folder_id = drive_folder
zip_name = CPT_PATH
directory_name = '.'
d = CPT_PATH

os.chdir(os.path.dirname(d))
with zipfile.ZipFile(d + '.zip',
                     "w",
                     zipfile.ZIP_DEFLATED,
                     allowZip64=True) as zf:
    for root, _, filenames in os.walk(os.path.basename(d)):
        for name in filenames:
            name = os.path.join(root, name)
            name = os.path.normpath(name)
            foo = zf.write(name, name)


In [0]:
file = drive.CreateFile({'parents':[{u'id': folder_id}]})
file.SetContentFile("checkpoint.zip")
file.Upload() 
print("zip file saved on Drive")